In [1]:
from __future__ import print_function
%matplotlib tk
from ReductionImports import *
import  NicoleNative as nic
setrcParams()
# dir_init = getFileFilesDir(2)
dir_init = '/media/hp/Data/CaII Polarimeter v2/Level-2'
print(dir_init)

/media/hp/Data/CaII Polarimeter v2/Level-2


In [2]:
#-------------------------- For KKL data
stokes_file = getFileFilesDir(0, initialdir = dir_init, 
                                 filetypes  = (('FITS file', '*STOKES*.fits'),), 
                                 title = 'Choose the stack of Stokes files...') # Select the file
hdu = pf.open(stokes_file) # Open with fits package
stokes_obs = float32(hdu[0].data) # Read the data
header = hdu[0].header # Read the header
hdu.close() # Clos HDU
print(stokes_file, ' is loaded')
ns, nw, ny, nx = stokes_obs.shape
#---------------------------------------------------- Fixed Parameters
pix_caii = header['PIXCAII']
pix_fei = header['PIXFEI']
pix_sii = header['PIXSII']
wav_caii = 8542.089
wav_atm1 = 8540.8
wav_fei = 8538.015
wav_sii = 8536.165
plate_scale = 0.07425*int(header['BINNING'])*int(header['IMBIN']) # Plate scale in arcsec/pixel
lin_disp = header['DISPERS']
nw_ran = (arange(nw) - pix_fei)*lin_disp + wav_fei
#---------------------------------------------------- Parameters for inversion
wav_n = 80
pix_start = int(pix_fei)-wav_n/2
wav_ran = nw_ran[pix_start:pix_start+wav_n]

/media/hp/Data/CaII Polarimeter v2/Level-2/20180426_085155_STOKESCORRECT.fits  is loaded


In [6]:
# For synthesis mode
model_in = 'fei-8538-mod.model'
model_interp = 'fei-8538-interp.mod'
profile_synth = 'fei-8538synth.pro'
# For inversion mode
model_guess = 'falc.model'
model_out = 'fei-8538-fitted.mod'
profile_obs = 'fei-8538-obs.pro'
profile_out = 'fei-8538-fitted.pro'
x_bin, y_bin = 4,4
x_beg, x_end, y_beg, y_end = get_xy(stokes_obs[0,0], x_bin, y_bin)
print(x_beg, x_end, y_beg, y_end)

35 39 53 57


In [4]:
cont_beg = int(pix_fei) - 52
cont_end = cont_beg + 92
cont_reg = median(stokes_obs[0,cont_beg:cont_end], axis=(1,2))
cont_wav = nw_ran[cont_beg:cont_end]
cont_w = cont_reg-cont_reg.min()
cont_w[32:80] = 0
p = polyfit(cont_wav, cont_reg, 1, w=cont_w)
cont_fit = p[0]*cont_wav + p[1]
plot(cont_reg)
figure()
plot(cont_wav, cont_reg, cont_wav, cont_fit)

In [7]:
stokes_temp = mean(stokes_obs[:,pix_start:pix_start+wav_n, y_beg:y_end, x_beg:x_end], axis=(2,3))
cont_corr = p[0]*wav_ran + p[1]
cont_corr /= cont_corr.max()
stokes_temp /=  cont_corr
stokes_temp /= stokes_temp[0,0]
stokes_temp = gaussian_filter1d(stokes_temp, 0.001)
stokes_temp[1] -= 0.016*stokes_temp[3]-0.001*stokes_temp[0]
stokes_temp[2] += -0.02*stokes_temp[3]-0.001*stokes_temp[0]
stokes_temp[2,:] = -stokes_temp[2,:] 
stokes_temp[3,:] = stokes_temp[3,:]-0.003*stokes_temp[0]
nic.plot_profile(figure(0), [stokes_temp])
observ = transpose(append(reshape(wav_ran, (1, wav_n)) , stokes_temp, axis=0))
savetxt(profile_obs, observ, fmt='{:^10}'.format('%s'), delimiter=' ')

In [8]:
# New model
me_pars, me_head, me_const, me_out = nic.read_ascii_model('valc.model')
me_const = ' ' + str(int((lin_disp*3/8538)*2.998*10**10)) + ' 0.0\r\n'
N = me_out.shape[0]
me_out[:,4] = linspace(1500, 500, N) # b_los
me_out[:,5] = linspace(150000, 50000, N) # v_los
me_out[:,6] = linspace(-400, -800, N) # b_x
me_out[:,7] = linspace(-300, -600, N) # b_y
nic.create_ascii_model(model_in, me_head, me_const, me_out)

# Invert

In [10]:
INPUT = nic.set_nicole_input_params(Mode = 'I',
                                    Printout_detail = 1,
                                    Input_model = model_in,
                                    Observed_profiles = profile_obs,
                                    Output_profiles = profile_out,
                                    Heliocentric_angle = 0.97, 
                                    Output_model = model_out,
                                    noise_level = 0.002,
                                    Maximum_number_of_inversions=5,
                                    Continuum_reference = 3,
                                    Debug_mode = 1)
INPUT = nic.set_nicole_regions_lines(INPUT, 'Region 1',
                                     First_wavelength = wav_ran[0], 
                                     Wavelength_step = lin_disp,
                                     Number_of_wavelengths = wav_n)
INPUT = nic.set_nicole_regions_lines(INPUT, 'Line 1',
                                     Line = 'FeI_8538',
                                     Mode = 'LTE')
INPUT = nic.set_nicole_regions_lines(INPUT, 'Nodes',
                                     Temperature = 4,
                                     Velocity = 2,
                                     Bz = 2,
                                     By = 1,
                                     Bx = 1)

F = open('NICOLE.input', 'w')
F.write(INPUT)
F.close()
try:
    os.remove(profile_out)
    os.remove(model_out)
except: pass

In [9]:
mi_pars, mi_head, mi_const, mi_plot = nic.read_ascii_model(model_in)
mo_pars, mo_head, mo_plot, mo_const, mo_abu = nic.read_native_model(model_out)
nic.plot_model_bvtp(figure(3), [mi_plot, mo_plot])
s_out = nic.read_native_profile([profile_out])
nic.plot_profile(figure(4), [stokes_temp])
s_out = [gaussian_filter1d(s_out[0], 3, axis=1)]
nic.plot_profile(figure(4), s_out)

nicole2.3bp     
1 1 80


/home/hp/miniconda2/lib/python2.7/site-packages/matplotlib/cbook/deprecation.py:107: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [166]:
INPUT = nic.set_nicole_input_params(Mode = 'S',
                                Input_model = model_in ,
                                Output_profiles = profile_synth,
                                Heliocentric_angle = 1, 
                                Output_model = model_interp,
                                Continuum_reference = 4,
                                Debug_mode = 1)
INPUT = nic.set_nicole_regions_lines(INPUT, 'Region 1',
                                 First_wavelength = wav_ran[0], 
                                 Wavelength_step = lin_disp,
                                 Number_of_wavelengths = wav_n,
                                 Macroturbulence_enhancement = 1)
INPUT = nic.set_nicole_regions_lines(INPUT, 'Line 1',
                                 Line = 'CaII_8542')
F = open('NICOLE.input', 'w')
F.write(INPUT)
F.close()
try:
    os.remove(profile_synth)
except: pass

In [ ]:
# For synthesis mode
model_in = 'mymodel.model'
model_interp = 'caii-8542-interp.mod'
profile_synth = 'caii-8542-synth.pro'
# For inversion mode
model_guess = 'falc.model'
model_out = 'caii-8542-fitted.mod'
profile_obs = 'caii-8542-obs.pro'
profile_out = 'caii-8542-fitted.pro'
# New model
me_pars, me_head, me_const, me_out = nic.read_ascii_model('valc.model')
me_const = ' ' + str(int((lin_disp*0.0/8538)*2.998*10**10)) + ' 0.0\r\n'
N = me_out.shape[0]
me_out[:,4] = linspace(-3500, -1500, N) # b_los
me_out[:,5] = linspace(250000, 350000, N) # v_los
me_out[:,6] = linspace(500, 1500, N) # b_x
me_out[:,7] = linspace(-500, -1500, N) # b_y
nic.create_ascii_model(model_in, me_head, me_const, me_out)

In [ ]:
%run '/home/hp/NICOLE/run/run_nicole.py'
close()

In [ ]:
mi_pars, mi_head, mi_const, mi_plot = nic.read_ascii_model(model_in)
mt_pars, mt_head, mt_plot, mt_const, mt_abu = nic.read_native_model(model_interp)
nic.plot_model_bvtp(figure(1), [mi_plot, mt_plot])
s_synth = nic.read_native_profile([profile_synth])
s_synth = [gaussian_filter1d(s_synth[0], 2, axis=1)]
nic.plot_profile(figure(2), s_synth + [stokes_temp])

# Create instrument profile

In [ ]:
def gaussian_1d_func(x, a, mu, sig):
    return a*exp(-(x-mu)**2/(2*sig**2))
instru_resp = gaussian_1d_func(wav_ran, 1, wav_ran[0], 2*lin_disp) + gaussian_1d_func(wav_ran, 1, wav_ran[-1], 2*lin_disp)

In [ ]:
profile_instru = 'Instrumental_profile.dat'
instru = append(reshape(wav_ran, (1, wav_n)) , ones([4,1])*reshape(instru_resp, (1, wav_n)), axis=0)
# instru[2::] = 0
# instru = reshape(instru_resp, (1, wav_n))
instru = transpose(instru)
savetxt(profile_instru, instru, fmt='{:^10}'.format('%s'), delimiter=' ')
plot(instru)

In [27]:
weights = np.copy(observ)
weights[:,1::] = 5
weights[40:74, 1] = 1.0/10000
weights[43:-86, 2] = 1.0/10
weights[43:86, 3] = 1.0/500
weights[:,4] = 1.0/1000
savetxt('Weights.pro', weights, fmt='{:^10}'.format('%s'), delimiter=' ')

## ZIMPOL3 data

In [124]:
#--------------------- For ZIMPOL3 data
stokes_file = getFileFilesDir(0, initialdir = dir_init, 
                                 filetypes  = (('FITS file', '*STOKES*.fits'),), 
                                 title = 'Choose the stack of Stokes files...') # Select the file
hdu = pf.open(stokes_file) # Open with fits package
stokes_obs = float32(hdu[0].data) # Read the data
# header = hdu[0].header # Read the header
hdu.close() # Clos HDU
print(stokes_file, ' is loaded')
ns, nw, ny, nx = stokes_obs.shape
pix_caii = 602
pix_fei = 296
pix_sii = 158.5
wav_caii = 8542.089
wav_atm1 = 8540.8
wav_fei = 8538.015
wav_sii = 8536.165
plate_scale = 0.07425*int(header['BINNING'])*int(header['IMBIN']) # Plate scale in arcsec/pixel
lin_disp = 0.0133
nw_ran = (arange(nw) - pix_fei)*lin_disp + wav_fei
#---------------------------------------------------- Parameters for inversion
wav_n = 120
pix_start = int(pix_fei)-wav_n/2
wav_ran = nw_ran[pix_start:pix_start+wav_n]

/media/hp/Data/CaII Polarimeter v2/Level-2/20180426_085155_STOKESCORRECT.fits  is loaded
